## importing libraries

In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

## importing data from csv file

In [14]:
df=pd.read_csv(r"C:\Users\jagad\Desktop\mosquito dashboard\data board\Key West_Data_updated.csv")

## sample of data

In [13]:
df.head()

,OID,YEAR,WEEK,Month,Nmonth,ADDRESS,Longitude,Latitude,Ae_aegypti_femalers,Ae_aegypti_males,Cx_quinquefasciatus_females,Cx_quinquefasciatus_males
0,53,2010,1,December,12,"1121 MARGARET, Key West, FL,",-81.7937,24.552867,ND,ND,ND,ND
1,54,2010,2,January,1,"1121 MARGARET, Key West, FL,",-81.7937,24.552867,6,1,0,0
2,55,2010,3,January,1,"1121 MARGARET, Key West, FL,",-81.7937,24.552867,1,1,95,0
3,56,2010,4,January,1,"1121 MARGARET, Key West, FL,",-81.7937,24.552867,ND,ND,ND,ND
4,57,2010,5,January,1,"1121 MARGARET, Key West, FL,",-81.7937,24.552867,ND,ND,ND,ND


In [4]:
df.columns

Index(['OID', 'YEAR', 'WEEK', 'Month', 'Nmonth', 'ADDRESS', 'Longitude',
       'Latitude', 'Ae_aegypti_femalers', 'Ae_aegypti_males',
       'Cx_quinquefasciatus_females', 'Cx_quinquefasciatus_males'],
      dtype='object')

## filling missing values with 0 and converting numbers to numeric data

In [4]:
df_no_na=df.copy()
df2=df.copy()

df_no_na=df_no_na['Ae_aegypti_femalers']=='ND'
df2.loc[df_no_na,'Ae_aegypti_femalers']=0

df_no_na=df2['Ae_aegypti_males']=='ND'
df2.loc[df_no_na,'Ae_aegypti_males']=0

df_no_na=df2['Cx_quinquefasciatus_females']=='ND'
df2.loc[df_no_na,'Cx_quinquefasciatus_females']=0

df_no_na=df2['Cx_quinquefasciatus_males']=='ND'
df2.loc[df_no_na,'Cx_quinquefasciatus_males']=0

 #df2['Ae_aegypti_femalers'].astype(object).astype(int)
df2['Ae_aegypti_femalers'] = pd.to_numeric(df2['Ae_aegypti_femalers'])
df2['Ae_aegypti_males'] = pd.to_numeric(df2['Ae_aegypti_males'])
df2['Cx_quinquefasciatus_females'] = pd.to_numeric(df2['Cx_quinquefasciatus_females'])
df2['Cx_quinquefasciatus_males'] = pd.to_numeric(df2['Cx_quinquefasciatus_males'])




## creating interactive dataframe and filters

In [5]:
#making data frame pipeline interactive
idf=df2.interactive()

#defining panel widgets
year_slider = pn.widgets.IntSlider(name='year slider', start=2010, end=2020, step=1)

mos_pipeline_afemale=(
    idf[(idf.YEAR==year_slider)]
    .groupby(['YEAR','Month','Nmonth'])['Ae_aegypti_femalers'].sum()
    .to_frame()
    .reset_index()
    .sort_values(by=['Nmonth','YEAR'])
    .reset_index(drop=True)
)

mos_pipeline_cmale=(
    idf[(idf.YEAR==year_slider)]
    .groupby(['YEAR','Month','Nmonth'])['Cx_quinquefasciatus_males'].sum()
    .to_frame()
    .reset_index()
    .sort_values(by=['Nmonth','YEAR'])
    .reset_index(drop=True)
)

mos_pipeline_cfemale=(
    idf[(idf.YEAR==year_slider)]
    .groupby(['YEAR','Month','Nmonth'])['Cx_quinquefasciatus_females'].sum()
    .to_frame()
    .reset_index()
    .sort_values(by=['Nmonth','YEAR'])
    .reset_index(drop=True)
)
mos_pipeline_amale=(
    idf[(idf.YEAR==year_slider)]
    .groupby(['YEAR','Month','Nmonth'])['Ae_aegypti_males'].sum()
    .to_frame()
    .reset_index()
    .sort_values(by=['Nmonth','YEAR'])
    .reset_index(drop=True)
)

mos_plotaf = mos_pipeline_afemale.hvplot(x='Month', y='Ae_aegypti_femalers', line_width=2, title="mosquito population of Ae_aegypti_femalers")
mos_plotam = mos_pipeline_amale.hvplot(x='Month', y='Ae_aegypti_males', line_width=2, title="mosquito population of Ae_aegypti_males")
mos_plotcm = mos_pipeline_cmale.hvplot(x='Month', y='Cx_quinquefasciatus_males', line_width=2, title="mosquito population of Cx_quinquefasciatus_males")
mos_plotcf = mos_pipeline_cfemale.hvplot(x='Month', y='Cx_quinquefasciatus_females', line_width=2, title="mosquito population of Cx_quinquefasciatus_females")


## map which shows the graph of mosquiotes

In [7]:
import pydeck
import panel as pn
import pydeck as pdk
pn.extension('deckgl')

DATA_URL = "https://raw.githubusercontent.com/uber-common/deck.gl-data/master/examples/geojson/vancouver-blocks.json"
MAPBOX_KEY = "pk.eyJ1IjoicGFuZWxvcmciLCJhIjoiY2s1enA3ejhyMWhmZjNobjM1NXhtbWRrMyJ9.B_frQsAVepGIe-HiOJeqvQ"

geojson_tooltip = {
    "html": """
      <b>Value per Square meter:</b> {properties.valuePerSqm}<br>
      <b>Growth:</b> {properties.growth}
    """,
    "style": {
        "backgroundColor": "steelblue",
        "color": "white"
    }
}

tooltip = {
    "geojson": geojson_tooltip,
    "landcover": {
        "html": "The background",
        "style": {
            "backgroundColor": "red",
            "color": "white"
       }
    }
}

# pn.pane.DeckGL(json_spec, tooltips=tooltip, mapbox_api_key=MAPBOX_KEY, sizing_mode='stretch_width', height=600)


LAND_COVER = [[[-123.0, 49.196], [-123.0, 49.324], [-123.306, 49.324], [-123.306, 49.196]]]

INITIAL_VIEW_STATE = pydeck.ViewState(
  latitude=49.254,
  longitude=-123.13,
  zoom=11,
  max_zoom=16,
  pitch=45,
  bearing=0
)

polygon = pydeck.Layer(
    'PolygonLayer',
    LAND_COVER,
    stroked=False,
    # processes the data as a flat longitude-latitude pair
    get_polygon='-',
    get_fill_color=[0, 0, 0, 20]
)

geojson = pydeck.Layer(
    'GeoJsonLayer',
    DATA_URL,
    opacity=0.8,
    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    get_elevation='properties.valuePerSqm / 20',
    get_fill_color='[255, 255, properties.growth * 255]',
    get_line_color=[255, 255, 255],
    pickable=True
)

r = pydeck.Deck(
    api_keys={'mapbox': MAPBOX_KEY},
    layers=[polygon, geojson],
    initial_view_state=INITIAL_VIEW_STATE
)

# Tooltip (you can get the id directly from the layer object)
tooltips = {geojson.id: geojson_tooltip}
r.to_html("column_layer_5.html")
jhg= pn.pane.DeckGL(r, sizing_mode='stretch_width', tooltips=tooltips, height=600)

## creating the dashboard layout available on local host

In [8]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Mosquito Population in KeyWest, Florida', 
    sidebar=[pn.pane.Markdown("# 4 Kinds of mosqitos are being considered in this dashboard"), 
             pn.pane.Markdown("Aedes aegypti, the yellow fever mosquito, is a mosquito that can spread dengue fever, chikungunya, Zika fever, Mayaro and yellow fever viruses, and other disease agents."), 
             pn.pane.GIF('https://api.time.com/wp-content/uploads/2016/02/zika-transmission-step1.gif', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"),   
             year_slider],
    main=[pn.Row(pn.Column(mos_plotaf.panel(width=700), margin=(0,25)), 
                 mos_plotam.panel(width=700)), 
          pn.Row(pn.Column(mos_plotcf.panel(width=700), margin=(0,25)), 
                 pn.Column(mos_plotcm.panel(width=700))),
          pn.Row(jhg)],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();

In [10]:
import pydeck
import panel as pn
import pydeck as pdk
pn.extension('deckgl')

#defining panel widgets
import panel as pn
import pandas as pd
import pydeck as pdk
import hvplot.pandas
idf=df2.interactive()
pn.extension('tabulator')


from ipywidgets import interact, interactive,fixed,interact_manual
typer= species_list
k=interact(call_me, yearsd=(2010,2020,1),typed=typer)

NameError: name 'species_list' is not defined

## generates a map on mosquito dash board
#### it also creates a seperate HTML file of the map with mosquito population as bar

In [11]:
def call_me(yearsd,typed):
    try_df=(
    df2[(df2.YEAR==yearsd)]
    .groupby(['Longitude','Latitude'])[typed].sum()  #'Ae_aegypti_femalers'
    .to_frame()
    .reset_index()
    .sort_values(by=['Longitude'])
    .reset_index(drop=True))
    print("function called")

    view_state = pdk.ViewState(
        longitude=-81.79369981,
        latitude=24.55286657,
        zoom=12,
        min_zoom=5,
        max_zoom=15,
        pitch=40.5,
        bearing=-27.36)

    column_layer = pdk.Layer(
        "ColumnLayer",
        data=try_df,
        get_position=["Longitude", "Latitude"],
        get_elevation=typed,   #"Ae_aegypti_femalers"
        elevation_scale=10,
        radius=75,

        get_fill_color=[180, 0, 200, 140],
        pickable=True,
        auto_highlight=True,
    )

    tooltip = {
        "html": "<b>{mrt_distance}</b> meters away from an MRT station, costs <b>{price_per_unit_area}</b> NTD/sqm",
        "style": {"background": "grey", "color": "white", "font-family": '"Helvetica Neue", Arial', "z-index": "10000"},
    }

    r = pdk.Deck(
        column_layer,
        initial_view_state=view_state,


    )

    return r.to_html("column_layer.html")

In [15]:
from ipywidgets import interact, interactive,fixed,interact_manual
typer=['Ae_aegypti_femalers','Ae_aegypti_males','Cx_quinquefasciatus_females','Cx_quinquefasciatus_males ']
k=interact(call_me, yearsd=(2010,2020,1),typed=typer)

interactive(children=(IntSlider(value=2015, description='yearsd', max=2020, min=2010), Dropdown(description='t…